
<img src="../assets/blocks_animation_v3.gif" align="center" width="100%" margin="0" ></img> 
<br>

# `DEMOLITION PREMONITION`
# - 
##### `by Samuel He`
---
## 02 Scraping OneMapSG's API
---



In [1]:
import requests
import time
import pandas as pd
import numpy as np
from random import randint
from onemapsg import OneMapClient
from random import randint
from geopy.distance import lonlat, distance
from math import sin, cos, sqrt, atan2, radians

# UNCOMMENT TO INSTALL GEOPY
#!pip3 install --upgrade --target=/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages geopy

# UNCOMMENT TO INSTALL THE ONEMAPSG API WRAPPER
# !pip3 install --upgrade --target=/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages onemapsg

In [2]:
#READING IN CSVs

demo_blocks_clean = pd.read_csv("../data/demo_blocks_new_21April_5pm.csv")


In [41]:
stand_blocks_clean = pd.read_csv("../data/stand_blocks_new_21April_5pm.csv")

---
### 2.1 Exploring the API

We know that we can obtain location, postal code and long-lat data from the OneMapSG API. So, let's do that first. 

In [3]:
#INPUT PASSWORD AND USERNAME
#Client = OneMapClient("samuelhezhengbang@gmail.com", "**PASSWORD REDACTED**")

In [4]:
#WE'RE GOING TO START WITH DEMOLISHED BLOCKS FIRST AS I ASSUME THAT THAT'S GOING TO BE TOUGH!
demo_blocks_clean.head(2)

,latest_survival_date,year_completed,blk_no,street,max_floor_lvl,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,other_sold,is_demolished
0,1995-08-22,1952.0,2,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,1
1,1995-08-22,1952.0,4,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,1


In [5]:
#TRYING MY LUCK WITH GETTING LOCATIONS FOR OUR DEMOLISHED BLOCKS
#SOME DEMOLISHED BLOCKS HAVE REINCARNATED AS NEW BLOCKS! THAT'S GOOD!
pd.DataFrame(Client.search("519 WEST COAST ROAD")["results"])

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,WEST COAST PRIDE,519,WEST COAST ROAD,WEST COAST PRIDE,519 WEST COAST ROAD WEST COAST PRIDE SINGAPORE...,120519,19769.64262,32220.491110000003,1.3076640959999999,103.75936490000001,103.75936490000001


---
### 2.1.2 Creating a function to extract location data from OneMapSG
With a convenient function, we can always return to this file with more data in the future. But for today, it'll help us get location data from both datasets.

In [25]:
#CREATING A FUNCTION TO EXTRACT LOCATION DETAILS FROM OneMapSG
def extract_address_data(ref_df):
    #CREATING VALUES FOR A SITUATION WHERE THERE IS NO LOCATION FOUND
    empty_col =[{'SEARCHVAL': "NaN",
 'BLK_NO': "NaN",
 'ROAD_NAME': "NaN",
 'BUILDING': "NaN",
 'ADDRESS': "NaN",
 'POSTAL': "NaN",
 'X': "NaN",
 'Y': "NaN",
 'LATITUDE': "NaN",
 'LONGITUDE': "NaN",
 'LONGTITUDE': "NaN"}]
    #CREATING COLUMNS FOR THE DATAFRAME
    api_cols = ['SEARCHVAL',
     'BLK_NO',
     'ROAD_NAME',
     'BUILDING',
     'ADDRESS',
     'POSTAL',
     'X',
     'Y',
     'LATITUDE',
     'LONGITUDE',
     'LONGTITUDE']
    #CREATING A DATAFRAME
    output_df = pd.DataFrame(columns = api_cols)
    #CREATING EMPTY LISTS TO NEST INCOMPLETE SEARCHES
    no_location = []
    no_street = []
    #FOR LOOP THROUGH EVERY ROW IN CHOSEN DATASET
    for row in range(ref_df.shape[0]):
        #CREATING QUERY STRINGS FOR THE API WRAPPER
        query_string_detailed = str(ref_df.iloc[row]["blk_no"] + " "+ (ref_df.iloc[row]["street"]).upper().split("/")[0])
        query_string_street =  str((ref_df.iloc[row]["street"]).upper().split("/")[0])
        
        #THESE ARE EXCEPTIONS. 
        #THAT IS: THESE STREETS DON'T EXIST ANYMORE
        #PLS TUNE THESE FOR EVERY DATASET
        if query_string_street == 'SILAT WALK ' :
            query_string_street = "SILAT AVE"
        if query_string_street == "NILE RD":
            query_string_street = "HAVELOCK VIEW"
        if query_string_street == 'JLN MEMBINA BARAT ':
            query_string_street = "JLN MEMBINA"
        if query_string_street == 'FORFAR HOUSE':
            query_string_street = "STRATHMORE AVE"      
        if query_string_street == 'TRONOH RD':
            query_string_street = "BOON KENG RD"            
        if query_string_street == 'JLN PASAR BARU ':
            query_string_street = "GEYLANG SERAI"          
            
        #CONDITIONS IF SEARCH IS UNFUFILLED
        if Client.search(query_string_detailed)["results"]==[]:
            if Client.search(query_string_street)["results"]==[]:
                #INPUT A ROW OF NaNs SHOULD WE NOT FIND ANYTHING
                output_df=output_df.append(empty_col)
                no_location.append(query_string_street)
            else: 
                #"PEPPER" SCRIPT: TO AVOID ONLY SHOWING THE SAME BLOCK WE ARE GOING TO RANDOMISE
                if (len(Client.search(query_string_street)["results"]))>1:
                    random_pick = randint(0, (len(Client.search(query_string_street)["results"])-1) )
                else:
                    random_pick = 0
                #COMPROMISE MADE: WE CAN'T FIND THE BLOCK, BUT WE'LL PUT GENERAL LOCATION DETAILS
                output_df=output_df.append((Client.search(query_string_street)["results"][random_pick]), ignore_index=True)
                no_street.append(query_string_detailed)
        
        #BEST CASE SCENARIO. BLOCK EXISTS ON OneMapSG
        else:
            output_df=output_df.append((Client.search(query_string_detailed)["results"][0]),ignore_index=True )
    #PRINTING STATUS REPORTS        
    print("Location not found:{}\n".format(no_location))
    print("Specific Location not found, General Location found:{}".format(no_street))
    if output_df.shape[0] == ref_df.shape[0]:
        print("\n\nWe managed to get entries for every row in your reference dataset")
    else:
        print("Opps, something went wrong :(")
    #RETURN A DATAFRAME
    return output_df

---
### 2.1.3 Extract location data for demo_blocks_clean


In [26]:
#CALLING THE FUNCTION ON demo_blocks_clean
demo_locations = extract_address_data(ref_df=demo_blocks_clean)
%time


Location not found:[]

Specific Location not found, General Location found:['2 BOON TIONG RD ', '4 BOON TIONG RD ', '6 BOON TIONG RD ', '8 BOON TIONG RD ', '12 BOON TIONG RD ', '13 BOON TIONG RD ', '1 BOON TIONG RD ', '3 BOON TIONG RD ', '5 BOON TIONG RD ', '7 BOON TIONG RD ', '9 BOON TIONG RD ', '14 BOON TIONG RD ', '15 BOON TIONG RD ', '151 LOR 2 TOA PAYOH', '107 DEPOT RD', '67 REDHILL RD', '6a MARGARET DR', '6b MARGARET DR', '39 FORFAR HOUSE', '6 CHAI CHEE RD', '7 CHAI CHEE RD', '309 ANG MO KIO AVE 1', '315 ANG MO KIO AVE 1', '316 ANG MO KIO AVE 1', '139 CORPORATION DR ', '142 CORPORATION DR ', '143 CORPORATION DR ', '89 TOA PAYOH LOR 2', '454 ANG MO KIO AVE 10', '455 ANG MO KIO AVE 10', '51 NILE RD', '53 NILE RD', '54 NILE RD', '580 ANG MO KIO AVE 3', '581 ANG MO KIO AVE 3', '582 ANG MO KIO AVE 3', '24 DOVER CRES', '3 SYED ALWI RD', '4 SYED ALWI RD', '5 SYED ALWI RD', '135 TAH CHING RD', '136 TAH CHING RD', '137 TAH CHING RD', '138 TAH CHING RD', '136a ALEXANDRA RD ', '137a ALEXAND

In [27]:
demo_locations.head(3)

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,TIONG BAHRU VIEW,11A,BOON TIONG ROAD,TIONG BAHRU VIEW,11A BOON TIONG ROAD TIONG BAHRU VIEW SINGAPORE...,161011,27495.99207,29865.31242,1.286365781,103.8287899,103.8287899
1,BOON TIONG ARCADIA,8C,BOON TIONG ROAD,BOON TIONG ARCADIA,8C BOON TIONG ROAD BOON TIONG ARCADIA SINGAPOR...,166008,27589.5093,29933.988839999998,1.286986868,103.8296302,103.8296302
2,BOON TIONG ARCADIA,8A,BOON TIONG ROAD,BOON TIONG ARCADIA,8A BOON TIONG ROAD BOON TIONG ARCADIA SINGAPOR...,164008,27644.560439999997,29882.958619999998,1.28652537,103.83012480000001,103.83012480000001


In [28]:
demo_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SEARCHVAL   394 non-null    object
 1   BLK_NO      394 non-null    object
 2   ROAD_NAME   394 non-null    object
 3   BUILDING    394 non-null    object
 4   ADDRESS     394 non-null    object
 5   POSTAL      394 non-null    object
 6   X           394 non-null    object
 7   Y           394 non-null    object
 8   LATITUDE    394 non-null    object
 9   LONGITUDE   394 non-null    object
 10  LONGTITUDE  394 non-null    object
dtypes: object(11)
memory usage: 34.0+ KB


In [29]:
#CONCAT demo_locations TO demo_blocks_clean
demo_blocks_plus = demo_blocks_clean.join(demo_locations)
demo_blocks_plus.head(3)

,latest_survival_date,year_completed,blk_no,street,max_floor_lvl,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,...,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,1995-08-22,1952.0,2,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,...,11A,BOON TIONG ROAD,TIONG BAHRU VIEW,11A BOON TIONG ROAD TIONG BAHRU VIEW SINGAPORE...,161011,27495.99207,29865.31242,1.286365781,103.8287899,103.8287899
1,1995-08-22,1952.0,4,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,...,8C,BOON TIONG ROAD,BOON TIONG ARCADIA,8C BOON TIONG ROAD BOON TIONG ARCADIA SINGAPOR...,166008,27589.5093,29933.988839999998,1.286986868,103.8296302,103.8296302
2,1995-08-22,1952.0,6,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,...,8A,BOON TIONG ROAD,BOON TIONG ARCADIA,8A BOON TIONG ROAD BOON TIONG ARCADIA SINGAPOR...,164008,27644.560439999997,29882.958619999998,1.28652537,103.83012480000001,103.83012480000001
3,1995-08-22,1952.0,8,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,...,8C,BOON TIONG ROAD,BOON TIONG ARCADIA,8C BOON TIONG ROAD BOON TIONG ARCADIA SINGAPOR...,166008,27589.5093,29933.988839999998,1.286986868,103.8296302,103.8296302
4,1995-08-22,1952.0,10,boon tiong rd / tiong bahru rd,4,bm,24.0,0.0,0.0,24.0,...,10A,BOON TIONG ROAD,TIONG BAHRU VIEW,10A BOON TIONG ROAD TIONG BAHRU VIEW SINGAPORE...,160010,27589.06418,29864.503930000003,1.286358471,103.82962619999999,103.82962619999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2016-08-03,1979.0,519,west coast rd,12,cl,174.0,0.0,0.0,150.0,...,519,WEST COAST ROAD,WEST COAST PRIDE,519 WEST COAST ROAD WEST COAST PRIDE SINGAPORE...,120519,19769.64262,32220.491110000003,1.3076640959999999,103.75936490000001,103.75936490000001
390,2016-08-03,1979.0,520,west coast rd,12,cl,154.0,0.0,0.0,132.0,...,520,WEST COAST ROAD,WEST COAST PRIDE,520 WEST COAST ROAD WEST COAST PRIDE SINGAPORE...,120520,19798.96394,32163.39455,1.3071477440000001,103.75962840000001,103.75962840000001
391,2018-05-13,1968.0,81,macpherson lane,12,gl,130.0,0.0,115.0,11.0,...,81,MACPHERSON LANE,MACPHERSON VIEW,81 MACPHERSON LANE MACPHERSON VIEW SINGAPORE 3...,360081,33895.91236,34930.10183,1.332169382,103.8862967,103.8862967
392,2018-05-13,1968.0,82,macpherson lane,12,gl,94.0,0.0,13.0,80.0,...,82,MACPHERSON LANE,MACPHERSON VIEW,82 MACPHERSON LANE MACPHERSON VIEW SINGAPORE 3...,360082,33820.61621,34942.69949,1.332283325,103.88562009999998,103.88562009999998



### 2.1.4 Extract location data for stand_blocks_clean


In [42]:
#WE RAN THE FUNCTION ONCE AND FOUND THAT OneMapSG DOES NOT ABBREVIATE "SAINT"
#SO, WE ARE REPLACING "ST. GEORGE" WITH "SAINT GEORGE"
stand_blocks_clean["street"].replace({"st\. george's": "saint george's"}, regex=True, inplace=True)

In [43]:
#CHECKING IF ALL IS WELL
stand_blocks_clean[["blk_no","street"]][stand_blocks_clean["street"].str.contains("george")]

,blk_no,street
591,7,king george's ave
779,2,saint george's rd
829,3,saint george's rd
846,1,saint george's rd
1750,5,saint george's lane
1768,803,king george's ave
1782,804,king george's ave
1783,805,king george's ave
1784,807,king george's ave
1863,6,saint george's lane


In [44]:
stand_blocks_clean.head(3)

,latest_survival_date,year_completed,blk_no,street,max_floor_lvl,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,other_sold,is_demolished
0,2020-04-24,1937,69,seng poh lane,1,bm,0,0,0,0,0,0,0,0,0
1,2020-04-24,1937,59,eng hoon st,3,bm,42,0,0,8,34,0,0,0,0
2,2020-04-24,1937,78,moh guan ter,5,bm,70,0,6,29,35,0,0,0,0


In [45]:
#CALLING THE FUNCTION ON stand_blocks_clean
stand_locations = extract_address_data(ref_df=stand_blocks_clean)
%time
stand_locations.head(3)

Location not found:[]

Specific Location not found, General Location found:[]


We managed to get entries for every row in your reference dataset
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,69 SENG POH LANE SINGAPORE 160069,69,SENG POH LANE,NIL,69 SENG POH LANE SINGAPORE 160069,160069,27940.70203,29620.0998,1.284148169,103.8327858,103.8327858
1,TIONG BAHRU ESTATE,59,ENG HOON STREET,TIONG BAHRU ESTATE,59 ENG HOON STREET TIONG BAHRU ESTATE SINGAPOR...,160059,28013.1071,29620.08015,1.284147991,103.83343640000001,103.83343640000001
2,78 MOH GUAN TERRACE SINGAPORE 162078,78,MOH GUAN TERRACE,NIL,78 MOH GUAN TERRACE SINGAPORE 162078,162078,27685.482330000003,29563.16548,1.283633273,103.8304925,103.8304925


In [46]:
stand_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12157 entries, 0 to 12156
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SEARCHVAL   12157 non-null  object
 1   BLK_NO      12157 non-null  object
 2   ROAD_NAME   12157 non-null  object
 3   BUILDING    12157 non-null  object
 4   ADDRESS     12157 non-null  object
 5   POSTAL      12157 non-null  object
 6   X           12157 non-null  object
 7   Y           12157 non-null  object
 8   LATITUDE    12157 non-null  object
 9   LONGITUDE   12157 non-null  object
 10  LONGTITUDE  12157 non-null  object
dtypes: object(11)
memory usage: 1.0+ MB


In [48]:
#CONCAT stand_locations TO stand_blocks_clean
stand_blocks_plus = stand_blocks_clean.join(stand_locations)
stand_blocks_plus.head(3)

,latest_survival_date,year_completed,blk_no,street,max_floor_lvl,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,...,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,2020-04-24,1937,69,seng poh lane,1,bm,0,0,0,0,...,69,SENG POH LANE,NIL,69 SENG POH LANE SINGAPORE 160069,160069,27940.70203,29620.0998,1.284148169,103.8327858,103.8327858
1,2020-04-24,1937,59,eng hoon st,3,bm,42,0,0,8,...,59,ENG HOON STREET,TIONG BAHRU ESTATE,59 ENG HOON STREET TIONG BAHRU ESTATE SINGAPOR...,160059,28013.1071,29620.08015,1.284147991,103.83343640000001,103.83343640000001
2,2020-04-24,1937,78,moh guan ter,5,bm,70,0,6,29,...,78,MOH GUAN TERRACE,NIL,78 MOH GUAN TERRACE SINGAPORE 162078,162078,27685.482330000003,29563.16548,1.283633273,103.8304925,103.8304925


In [49]:
#SAVING THE CREATING DATAFRAME TO A CSV FILE (22 APR 2020) 
#COMMENTING OUT THE CODE TO PREVENT DOUBLE SAVES.
# stand_blocks_plus.to_csv('../data/stand_blocks_latlon_22April_1am.csv', index = False)
# demo_blocks_plus.to_csv('../data/demo_blocks_latlon_22April_1am.csv', index = False)